In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [15]:
file=r'./subway_raw.csv'
raw=pd.read_csv(file)
raw.head()

,사용일자,요일,노선명,역명,승차총승객수,하차총승객수,승하자총승객수,연월,월일,등록일자
0,2019-01-01,월,2호선,을지로4가,3862.0,3728.0,7590.0,2019-01,01-01,20190104
1,2019-01-01,월,3호선,을지로3가,8104.0,7554.0,15658.0,2019-01,01-01,20190104
2,2019-01-01,월,2호선,을지로입구,22478.0,21330.0,43808.0,2019-01,01-01,20190104
3,2019-01-01,월,2호선,시청,8381.0,6049.0,14430.0,2019-01,01-01,20190104
4,2019-01-01,월,6호선,동묘앞,8045.0,8504.0,16549.0,2019-01,01-01,20190104


In [16]:
rawmonth=raw.groupby(['요일','월일']).agg(승하자총승객수=('승하자총승객수','sum'))
rawmonth.reset_index(inplace=True)
rawmonth

,요일,월일,승하자총승객수
0,금,01-05,12092335.0
1,금,01-12,12532124.0
2,금,01-19,12484719.0
3,금,01-26,12668145.0
4,금,02-02,10136207.0
...,...,...,...
176,화,05-29,17165171.0
177,화,06-05,17634177.0
178,화,06-12,16846068.0
179,화,06-19,16690880.0


In [17]:
x1=rawmonth[rawmonth['요일']=='월']['승하자총승객수']
#x1=x1.sample(n=24)
x2=rawmonth[rawmonth['요일']=='화']['승하자총승객수']
#x2=x2.sample(n=24)
x3=rawmonth[rawmonth['요일']=='수']['승하자총승객수']
#x3=x3.sample(n=24)
x4=rawmonth[rawmonth['요일']=='목']['승하자총승객수']
#x4=x4.sample(n=24)
x5=rawmonth[rawmonth['요일']=='금']['승하자총승객수']
#x5=x5.sample(n=24)
x6=rawmonth[rawmonth['요일']=='토']['승하자총승객수']
#x6=x6.sample(n=24)
x7=rawmonth[rawmonth['요일']=='일']['승하자총승객수']
#x7=x7.sample(n=24)
# 일요일 데이터 개수가 25개 이므로 24개씩 랜덤추출

In [18]:
print(x1.shape,x2.shape,x3.shape,x4.shape,x5.shape,x6.shape,x7.shape)
# 각 그룹의 표본수 N < 30 이므로 정규성 검정 실시

(26,) (26,) (26,) (26,) (26,) (26,) (25,)


# 귀무가설 H0 : 데이터셋이 정규분포를 따른다.
# 대립가설 H1 : 데이터셋이 정규분포를 따르지 않는다.

In [20]:
# Shapiro 정규성 검정
import scipy.stats as stats
x1s=stats.shapiro(x1)
x2s=stats.shapiro(x2)
x3s=stats.shapiro(x3)
x4s=stats.shapiro(x4)
x5s=stats.shapiro(x5)
x6s=stats.shapiro(x6)
x7s=stats.shapiro(x7)
xn=[x1s,x2s,x3s,x4s,x5s,x6s,x7s]
for i in xn:
    print(i)
# 금요일 데이터를 제외하면 P < 0.05 이므로 정규성을 만족한다고 보기 어려움

ShapiroResult(statistic=0.47478795051574707, pvalue=1.4111077639711311e-08)
ShapiroResult(statistic=0.5479180812835693, pvalue=7.820348457698856e-08)
ShapiroResult(statistic=0.6048225164413452, pvalue=3.362119116445683e-07)
ShapiroResult(statistic=0.7258560657501221, pvalue=1.222758692165371e-05)
ShapiroResult(statistic=0.9253191947937012, pvalue=0.059997688978910446)
ShapiroResult(statistic=0.8849973082542419, pvalue=0.007352896966040134)
ShapiroResult(statistic=0.4845786690711975, pvalue=2.643943908253732e-08)


In [21]:
# 귀무가설 H0 : 목요일의 승차총승객수는 나머지 요일의 승차총승객수와 유의미한 차이가 없다.
# 대립가설 H1 : 목요일의 승차총승객수는 나머지 요일의 승차총승객수와 유의미한 차이가 있다.

In [10]:
# Kruskal-Wallis H test : 분산분석에 대응하는 비모수적 검정방법
from scipy.stats import kruskal
kruskal(x1,x2,x3,x4,x5,x6,x7)
# P < 0.05 이므로 귀무가설 기각, 목요일은 다른 요일과 승차총승객수의 유의미한 차이가 있다.

KruskalResult(statistic=105.8951817413357, pvalue=1.4730475580839848e-20)

In [13]:
# 사후분석

# Nemenyi post-hoc test
    # pip install scikit-posthocs 필요
import scikit_posthocs as sp

x=np.array([x1,x2,x3,x4,x5,x6,x7])
sp.posthoc_nemenyi_friedman(x.T)
# 목요일은 3을 의미, 목요일은 금,토,일요일의 승차총승객수와 유의미한 차이가 존재한다.\

# 출퇴근이 많은 월,화,수,목요일에서 목요일의 승차총승객수가 가장 많은 것은
# 특정 사건 때문일 가능성이 존재하는 것으로 보인다.

,0,1,2,3,4,5,6
0,1.000000,0.900000,0.900000,0.169588,0.001570,0.00100,0.813735
1,0.900000,1.000000,0.900000,0.292768,0.001000,0.00100,0.656580
2,0.900000,0.900000,1.000000,0.499397,0.001000,0.00100,0.457177
3,0.169588,0.292768,0.499397,1.000000,0.001000,0.00100,0.002670
4,0.001570,0.001000,0.001000,0.001000,1.000000,0.65658,0.124446
5,0.001000,0.001000,0.001000,0.001000,0.656580,1.00000,0.001000
6,0.813735,0.656580,0.457177,0.002670,0.124446,0.00100,1.000000
